In [6]:
!pip install --upgrade pip

Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/fa/ee/74ff76da0ab649eec7581233daeb43d8aa35383d8f75317b2ab3b80c922f/pip-23.1.2.tar.gz
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/tmp/pip-build-rBb472/pip/setup.py", line 7
        def read(rel_path: str) -> str:
                         ^
    SyntaxError: invalid syntax
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-build-rBb472/pip/
You are using pip version 8.1.2, however version 23.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install --user kafka

You are using pip version 8.1.2, however version 23.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
!pip install pyspark==3.7

    100% |████████████████████████████████| 212.4MB 8.2kB/s  eta 0:00:01
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/tmp/pip-build-p_J6Q4/pyspark/setup.py", line 31
        file=sys.stderr)
            ^
    SyntaxError: invalid syntax
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-build-p_J6Q4/pyspark/
You are using pip version 8.1.2, however version 23.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
!pip install --user faker

  Using cached https://files.pythonhosted.org/packages/c3/97/d7f0b25e181738b24e25092acc160bf30c682da6ab961e28f839a5f862b2/Faker-18.11.2.tar.gz
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/tmp/pip-build-qFVSGa/faker/setup.py", line 3, in <module>
        from pathlib import Path
    ImportError: No module named pathlib
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-build-qFVSGa/faker/
You are using pip version 8.1.2, however version 23.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
pip install --user faker

     |████████████████████████████████| 1.6 MB 17.7 MB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("kafka_producer").getOrCreate()

In [3]:
# Producer
from kafka import KafkaProducer
from faker import Faker
import random
import time

# Kafka server and topic information
bootstrap_servers = 'w01.itversity.com:9092'
topic = 'MilkPro_Data_Pipeline1'

# Create a Kafka producer
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)

fake = Faker()

# Define the areas manually
areas = ["Karad", "Navarasta", "Patan", "Koyana"]

# Define farm IDs for each area
farm_ids = {
    "Karad": [f"KAR{id}" for id in range(1, 100)],
    "Navarasta": [f"NAV{id}" for id in range(101, 200)],
    "Patan": [f"PAT{id}" for id in range(201, 300)],
    "Koyana": [f"KOY{id}" for id in range(301, 400)]
}

# Global counter for data producer
data_counter = 0

# Generate and send 1000 rows of data every minute
for _ in range(1):
    for _ in range(1000):
        # Generate random milk properties
        id = fake.random_int(min=1, max=1000)

        # Generate random area
        area = random.choice(areas)

        # Select a farm ID from the list specific to the area
        farm_id = random.choice(farm_ids[area])

        # Generate timestamp within the last 3 months
        timestamp = fake.date_between(start_date='-3M', end_date='today').strftime('%d-%m-%Y')

        fat_percentage = format(fake.random.uniform(0.0, 9.0), '.2f')
        protein_percentage = format(fake.random.uniform(3.0, 4.2), '.2f')
        lactose_percentage = format(fake.random.uniform(4.5, 5.8), '.2f')
        temp = fake.random_int(min=50, max=100)

        # Generate Batch IDs for Morning and Evening procurement
        period = ["M", "E"]
        batch_ids = []
        for p in period:
            batch_id = f"{p}_{farm_id}_{timestamp}"
            batch_ids.append(batch_id)

        # Select a random batch ID
        batch_id = random.choice(batch_ids)
        
        #debug
        #data_id = data_counter + 1
        #data_counter += 1
        #print(f"-------------{data_id}---------")

        # Create a string message with the milk properties
        #message = f"{id},{fat_percentage},{protein_percentage},{lactose_percentage},{temp},{farm_id},{batch_id},{timestamp}"

        message = f"{id},{batch_id},{farm_id},{area},{timestamp},{fat_percentage},{protein_percentage},{lactose_percentage},{temp}"
        #print(message)

        # Send the message to Kafka
        producer.send(topic, message.encode('utf-8'))

    # Flush the messages to ensure they are sent immediately
    producer.flush()

    # Sleep for 2 minutes
    time.sleep(60)

# Close the Kafka producer
producer.close()